In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

from fastai.vision import *
from fastai.callbacks import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

import scipy.ndimage.measurements as meas
import scipy.ndimage as ndimage
import scipy.ndimage.morphology as mp
import skimage.morphology as mpg

### Load Model

In [ ]:
learn.load(f"path - ")

### Post-Processing & Submission

In [ ]:
result_csv_name = 'aimlab_submit.csv'

In [ ]:
with open(f'./{result_csv_name}.csv', 'w', encoding='utf-8', newline='') as file :
    wr = csv.writer(file)
    wr.writerow(['ImageId','EncodedPixels'])
    for path in test_img:
        a,b,c = learn.predict(open_image(path))
        d = np.argmax(c,axis=0)
        name = os.path.split(path)[-1][:-4]

        ### postprocessing
        ### label 분리
        gt1 = np.array(d).copy()
        gt2 = np.array(d).copy()
        gt3 = np.array(d).copy()

        all = np.array(d).copy()

        gt1[gt1!=1]=0
        gt2[gt2!=2]=0
        gt2[gt2==2]=1
        gt3[gt3!=3]=0
        gt3[gt3==3]=1

        all[all!=0]=1

        se2 = mpg.disk(20)
        gt_2 = mp.binary_closing(gt2,se2).astype('int')

        a = ndimage.binary_fill_holes(gt2+gt1+gt3).astype('int32') #3
        b = ndimage.binary_fill_holes(gt_2+gt1).astype('int32') #2
        c = ndimage.binary_fill_holes(gt1).astype('int32')#1

        c,num1 = meas.label(c)  
        unique1, counts1 = np.unique(c, return_counts=True)
        idxls = (-counts1).argsort()[:2]
        idx = idxls[1]
        #print(idx)
        c[c!=idx] = 0
        c[c==idx] = 1

        im1 = ((c-gt2)>0).astype('int') ##

        final2 = a+b+(im1)

        final2 = (final2 == 1).astype('int')*3+(final2 == 2).astype('int')*2+(final2 == 3).astype('int')

        for idx in range(1,4):
            np_array = np.where(final2 == idx,1,0)
            result = rle_encode(np_array).astype(str)
            result = ' '.join(result)
            wr.writerow([f'{name}_{idx}',result])